In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo

In [2]:
# Initializing pymongo to work w mongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
db = client.marsNews_db
collection = db.articles

In [4]:
#### MARS NEWS website

# page to be scraped
url = 'https://mars.nasa.gov/news'
# using requests module to retrieve page
response = requests.get(url)
# creating beautifulSoup object & parsing w html.parser
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup.prettify())

In [5]:
### MARS NEWS website

# Get header and subheader
marsNews_head = soup.find('div', class_='content_title').find('a').text.strip()
marsNews_subhead = soup.find('div', class_='slide').\
    find('div', class_='image_and_description_container').\
    find('a').find('div', class_='rollover_description').text.strip()
marsNews_link = soup.find('div', class_='slide').\
    find('div', class_='image_and_description_container').\
    find('a')['href']

print(marsNews_head)
print('------------')
print(marsNews_subhead)
print(url + marsNews_link)


NASA's Mars 2020 Rover Closer to Getting Its Name
------------
155 students from across the U.S. have been chosen as semifinalists in NASA's essay contest to name the Mars 2020 rover, and see it launch from Cape Canaveral this July.
https://mars.nasa.gov/news/news/8585/nasas-mars-2020-rover-closer-to-getting-its-name/


In [6]:
### Use SPLINTER to scrape jpl.nasa
from splinter import Browser

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [7]:
# Get main urls and visit the site w chrome driver

url_jpl = 'https://www.jpl.nasa.gov'
url_jpl_mars = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(url_jpl_mars)

In [8]:
# pull in beautiful soup to parse through the HTML

html_jpl = browser.html
soup = BeautifulSoup(html_jpl, 'html.parser')

In [9]:
# get the background image for getting featured image
# clean the string w replace to isolate the url

featured_img = soup.find('article')['style'].replace('background-image: url(','')
featured_img = featured_img.replace("'", "").replace(");", "")
featured_img

'/spaceimages/images/wallpaper/PIA19404-1920x1200.jpg'

In [10]:
# unite the html strings to get the full featured image url

featured_img_url = url_jpl + featured_img
featured_img_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19404-1920x1200.jpg'

In [11]:
########
####  Get mars weather data

mars_tweets = 'https://twitter.com/marswxreport'
response = requests.get(mars_tweets)
soup = BeautifulSoup(response.text, 'html.parser')

In [15]:
# get the latest weather tweet

mars_weather = soup.find('div', 'js-tweet-text-container').p.text
mars_weather = mars_weather.replace('\n', '').split('pic')
mars_weather = mars_weather[0]
mars_weather

'InSight sol 422 (2020-02-03) low -90.2ºC (-130.3ºF) high -13.1ºC (8.5ºF)winds from the SSE at 6.1 m/s (13.5 mph) gusting to 22.1 m/s (49.4 mph)pressure at 6.20 hPa'

In [16]:
###
import pandas as pd

space_facts_url = 'https://space-facts.com/mars/'
tables = pd.read_html(space_facts_url)
mars_df = tables[0]
mars_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [17]:
mars_df.columns = ['', 'Mars Planet Facts']
mars_df

,,Mars Planet Facts
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [18]:
## get mars hemispheres from USGS site

usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
usgs_response = requests.get(usgs_url)

In [19]:
# set up soup & splinter for visiting and searching
browser.visit(usgs_url)
usgs_html = browser.html
usgs_soup = BeautifulSoup(usgs_html, 'lxml')

In [20]:
# get the data for the four hemispheres 
hsphere_data = usgs_soup.find_all('div', class_='item')
hsphere_data

[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>,
 <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/7677c0a006b83871b5a2f66985ab5857_schiapa

In [21]:
# set variables for holding the hemisphere data and the main url for usgs astro

hspheres_img_urls = []
usgs_astro_url = 'https://astrogeology.usgs.gov'

In [22]:
# make a loop using isolated data above to pull out the headers and links
# ... and to store and append the data to the hsphere's list in a
# ... dictionary format

for h in hsphere_data:
    # getting header for the hemisphere
    title = h.find('h3').text
    
    # get the link ends for visiting the page
    link_end = h.find('a', class_='itemLink product-item')['href']
    
    # visit the particular hemisphere page
    browser.visit(usgs_astro_url + link_end)
    
    # set a new HTML object for the particular page
    link_end = browser.html
    
    # use soup to pull the data
    soup = BeautifulSoup(link_end, 'lxml')
    
    # use the home url and soup to concat the full image url
    h_img_url = usgs_astro_url + soup.find('img', class_='wide-image')['src']
    
    # append the full img url to the list
    hspheres_img_urls.append({'Title': title, 'Image URL': h_img_url})

hspheres_img_urls

[{'Title': 'Cerberus Hemisphere Enhanced',
  'Image URL': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'Title': 'Schiaparelli Hemisphere Enhanced',
  'Image URL': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'Title': 'Syrtis Major Hemisphere Enhanced',
  'Image URL': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'Title': 'Valles Marineris Hemisphere Enhanced',
  'Image URL': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]